# `H2MM_C` Secondary Control Features

Let's get our obligitory imports in order, and we'll load the 3 detector data as well.

In [1]:
import os
import numpy as np
from matplotlib import pyplot as plt

import H2MM_C as hm

# load the data
color3 = list()
times3 = list()

i = 0
with open('sample_data_3det.txt','r') as f:
    for line in f:
        if i % 2 == 0:
            times3.append(np.array([int(x) for x in line.split()],dtype='Q'))
        else:
            color3.append(np.array([int(x) for x in line.split()],dtype='L'))
        i += 1

## Optimization Control
Sometimes you want to control when optimizations stop and how many cores the optimization uses.

There are 4 distinct limits of this sort:
1. Number of cores- `num_cores = os.cpu_count // 2`
2. Maximum number of iterations- `max_iter = 3600`
3. Minimum number difference between loglik to consider converted- `converged_min = 1e-14`
4. Maximum time of optimizatoion- `max_time = np.inf`
    > **Note:**\
    > This counter uses the rather inaccurate C clock which tends to run fast, so your optimizations will often end earlier than the number entered.
    > The use of this parameter is generally discouraged

We'll start by demonstraing the use of `max_iter`.
Beging set by default to 3600, this is likely to be enough, but there is always the possibility that the optimization is still improving significantly.

Note that the optimization didn't converge when we optimized for 4 states with the 3 detector data.
So to increase the number of iterations, we use the keyword argument `max_iter` in `hm.EM_H2MM_C()`:

In [2]:
model_5s3d = hm.EM_H2MM_C(hm.factory_h2mm_model(4,3), color3, times3, max_iter=7200)

Optimization reached maximum number of iterations

The rest of the limits work in the same way, pass a keyword argument to `hm.EM_H2MM_C()` and over-ride the default.

Some notes about each of the limits:
- `num_cores` default is set when at import, where python calls `os.cpu_count() // 2` the reason that it uses `// 2` is because most machines are multi-threaded, and `os.cpu_count()` returns the number of cpu threads, and not the number of physical cores. However, model optimizations are cpu-intensive, so the ideal number of threads to use is the number of physical cores, not the nubmer of hyper-threads. If your machine is not hyper-threaded, you'll want to manually set this value. Another possibility is if you want to leave a cpu core or two open for other tasks, you could set `num_cores` to a smaller number.
- `max_iter` is the better way to limit the duration of optimizations. Make sure it is high enough, but often when an optimization does not converge quickly, it is because the model is over-fit and has too many states.
- `converged_min` sets how close to models have to be to consider a model converged. Due to floating point errors, it is entirely possible that two models with a very close loglik may improve only because of floating point error, or even for the "real" better value to be the oposite of what the calculation suggests. Especially if you have  a large data set, you might consider setting this to `1e-7` or similar value (small, but not as small as `1e-14`). It should be noted the differences in values between two models with such similar loglikelihoods will be negligable.
- `max_time` is by default infinite, so it is ignored. This is generaly best, as it uses an inaccurate C-clock, only use this if you really need to. The units are in seconds.

### Universal Defaults

To make it easier to change defaults, `H2MM_C` offers the `optimization_limits` variable, where you can change the default of the 4 optimization limits in the `H2MM_C.optimization_limits` variable, which uses the same syntax as a dictionary.

> This was inspired by the plt.rcParams varaible

So, for instance, if you know that the ideal number of cores is for instance 2, and not the default automatically supplied, instead of constantly setting the `num_cores` keyword, you can write:

`hm.optimization_limits['num_cores'] = 2`

At the beginning of the code, and not have to worry about the constantly writing the `num_cores` keyword argument.

This works for all the other 3 limits as well, so you can reduce the nubmer of iterations to 1000 with:

`hm.optimization_limits['max_iter'] = 1000`

Or, if you have many large and similarly sized data sets, and want to make difference needed to consider a model converged a bit larger, you can supply:

`hm.optimization_limits['converged_min'] = 1e-7`

While possible, as mentioned before, using `max_time` is generally discouraged.

In [3]:
hm.optimization_limits['num_cores'] = 2
hm.optimization_limits['max_iter'] = 1000
hm.optimization_limits['converged_min'] = 1e-7

model_5s3d = hm.EM_H2MM_C(hm.factory_h2mm_model(4,3), color3, times3)

Optimization reached maximum number of iterations

## Hashable models *New v2.0*

Version 2.0 introduced limited abilities to hash and use `h2mm_model`
objects as keys in dictionaries.

By default, most models cannot be used in this way,
rather a model must first be put into a canonical form.
To generate such a model, call the method `h2mm_model.sort_states`
on any existing model. The returned model will be hashable.

In [4]:
sorted_model_5s3d = model_5s3d.sort_states()
hash(sorted_model_5s3d)

4835756030495084663

Along with the hash function, h2mm_models can also be compared
to one another with the equality operator. When two models have
identical cannonical forms, they will evaluate to `True` in 
equality comparisons.

> **NOTE**
>
> This comparison is very strict, ie it is **not** an
> almost equal comparison.
> This is so that using hashable models as dictionary
> keys will still function.
> The floating point numbers in two models must be
> identical.

In [5]:
if sorted_model_5s3d == model_5s3d:
    print("sorteted model are equivalent to unsroted models")
else:
    print("sorteted model are not equivalent to unsroted models")
model_dummy = hm.factory_h2mm_model(4,3)
if model_dummy == model_5s3d:
    print("oops, unsorted models are comparable, and identical")
else:
    print("unsoreted models are comparable, but not identical")

sorteted model are equivalent to unsroted models
unsoreted models are comparable, but not identical


> **Note**
>
> Once a model is sorted so it can be hashed, it is fixed.
> It can no longer be assigned new `prior`/`trans`/`obs`
> values, and optimizations must be performed with `inplace=False`